In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dtoolbioimage import ImageDataSet, Image3D, Image
import numpy as np

In [ ]:
imageds = ImageDataSet('azure://jicimagedatasets/cc9d757a-b149-4626-83be-abc5a3765cb8')

In [ ]:
wall_stack = imageds.get_stack('fca-3_FLC-Venus_root01', 'fca-3_FLC-Venus_root01 #1', 1)

In [ ]:
venus_stack = imageds.get_stack('fca-3_FLC-Venus_root01', 'fca-3_FLC-Venus_root01 #1', 0)

In [ ]:
from dtoolbioimage.segment import sitk_watershed_segmentation, filter_segmentation_by_size

In [ ]:
from dtoolbioimage import zoom_to_match_scales
zoomed_stack = zoom_to_match_scales(wall_stack)

In [ ]:
zoomed_stack

In [ ]:
raw_segmentation = sitk_watershed_segmentation(zoomed_stack)

In [ ]:
raw_segmentation.view(Image3D)

In [ ]:
filtered = filter_segmentation_by_size(raw_segmentation)

In [ ]:
filtered.view(Image3D)

In [ ]:
from dtoolbioimage.segment import Segmentation3D

In [ ]:
uci = filtered.view(Segmentation3D).unique_color_image

In [ ]:
uci[:,:,10].view(Image)

In [ ]:
filtered.view(Segmentation3D).save('root01segmentation.tif')

In [ ]:
import os; os.getcwd()

In [ ]:
pretty = filtered.view(Segmentation3D).pretty_color_image

In [ ]:
pretty[:,:,10].view(Image)

In [ ]:
from dtoolbioimage.util.array import pretty_color_array
from ipywidgets import interactive, IntSlider, Output, HBox
from IPython.display import clear_output, display, display_png

def simple_segmentation_viewer(stack):

    _, _, max_z = stack.shape

    slider = IntSlider(min=0, max=max_z, step=1, description='Z plane:')

    def show_z_plane(z):
        display_png(pretty_color_array(stack[:,:,z]).view(Image))
    
    return interactive(show_z_plane, z=slider)


In [ ]:
simple_segmentation_viewer(filtered)

In [ ]:
from io import BytesIO
from imageio import imsave

In [ ]:
png_byte_arrays = []
for z in range(filtered.shape[2]):
    b = BytesIO()
    imsave(b, pretty_color_array(filtered[:,:,z]), 'PNG', compress_level=0)
    png_byte_arrays.append(b)

In [ ]:
class CachedImageView(object):
    
    def _repr_png_(self):
        return png_byte_arrays[0].getvalue()

In [ ]:
CachedImageView()

In [ ]:
from dtoolbioimage.util.array import pretty_color_array
from ipywidgets import interactive, IntSlider, Output, HBox
from IPython.display import clear_output, display, display_png

def cached_segmentation_viewer(stack):

    _, _, max_z = stack.shape
    
    slider = IntSlider(min=0, max=max_z, step=1, description='Z plane:')

    png_byte_arrays = {}
    def show_z_plane(z):
        if z in png_byte_arrays:
            raw_png_rep = png_byte_arrays[z]
        else:
            b = BytesIO()
            imsave(b, pretty_color_array(stack[:,:,z]), 'PNG', compress_level=0)
            raw_png_rep = b.getvalue()
            png_byte_arrays[z] = raw_png_rep

        display({'image/png': raw_png_rep}, raw=True)
    
    return interactive(show_z_plane, z=slider)

In [ ]:
cached_segmentation_viewer(filtered)

In [ ]:
filtered[np.where(filtered > 50)] = 0

In [ ]:
filtered.view(Segmentation3D)

In [ ]:
fpath = 'root01segmentation.tif'

In [ ]:
from imageio import volread

In [ ]:
unique_color_image = volread(fpath)

In [ ]:
unique_color_image.shape

In [ ]:
print(unique_color_image[18,256,512])

In [ ]:
unique_color = unique_color_image[18,256,512]

In [ ]:
from dtoolbioimage.util.color import identifier_from_unique_color

In [ ]:
identifier_from_unique_color(unique_color)

In [ ]:
planes = []
for z in range(unique_color_image.shape[0]):
    pass

In [ ]:
zdim, xdim, ydim, _ = unique_color_image.shape
planes = []
for z in range(zdim):
    segmentation = np.zeros((xdim, ydim), dtype=np.uint32)
    segmentation += unique_color_image[z,:,:,2]
    segmentation += unique_color_image[z,:,:,1] * 256
    segmentation += unique_color_image[z,:,:,0] * 256 * 256
    planes.append(segmentation)

In [ ]:
np.dstack(planes).view(Segmentation3D)

In [ ]:
Segmentation3D.from_file('root01segmentation.tif')

In [ ]:
fpath = 'root01segmentation.tif'

In [ ]:
unique_color_image = volread(fpath)

In [ ]:
unique_color_image.shape

In [ ]:
from skimage.measure import regionprops

In [ ]:
filtered.view(Segmentation3D)

In [ ]:
zoomed_venus_stack = zoom_to_match_scales(venus_stack)

In [ ]:
zoomed_venus_stack

In [ ]:
from skimage.measure import regionprops

In [ ]:
by_label = {r.label: r for r in regionprops(filtered)}

In [ ]:
by_label[5].bbox

In [ ]:
rmin, cmin, zmin, rmax, cmax, zmax = by_label[5].bbox

In [ ]:
selected = filtered[rmin:rmax, cmin:cmax, zmin:zmax]

In [ ]:
selected.view(Segmentation3D)

In [ ]:
region = selected == 5

In [ ]:
from scipy.ndimage.morphology import binary_erosion

In [ ]:
eroded = binary_erosion(region)

In [ ]:
eroded.view(Image3D)

In [ ]:
surface = region ^ eroded

In [ ]:
surface.view(Image3D)

In [ ]:
np.sum(surface)

In [ ]:
np.sum(region)

In [ ]:
def spherality(region):
    eroded = binary_erosion(region)
    surface = region ^ eroded
    
    S = np.sum(surface)
    V = np.sum(region)
    
    mult = 4.5 * np.sqrt(np.pi)
    
    return mult * V / np.power(S, 1.5)
    

In [ ]:
spherality(region)

In [ ]:
np.power(2, 1.5)

In [ ]:
from skimage.morphology import ball

In [ ]:
ball(20).view(Image3D)

In [ ]:
spherality(ball == 1)

In [ ]:
ballbool = ball(100) == 1

In [ ]:
ballbool.view(Image3D)

In [ ]:
eroded = binary_erosion(ballbool)

In [ ]:
surface = ballbool ^ eroded

In [ ]:
surface.view(Image3D)

In [ ]:
spherality(ballbool)

In [ ]:
ballbool.shape

In [ ]:
(4 / 3) * np.pi * np.power(100, 3)

In [ ]:
np.sum(ballbool)

In [ ]:
np.sum(surface)

In [ ]:
4 * np.pi * 100 * 100

In [ ]:
V = (4 / 3) * np.pi * np.power(100, 3)
S = 4 * np.pi * 100 * 100
mult = 6 * np.sqrt(np.pi)
mult * V / np.power(S, 1.5)

In [ ]:
from dtoolbioimage.segment import select_region, spherality

In [ ]:
r = select_region(filtered, 50)

In [ ]:
r.view(Image3D)

In [ ]:
wonky = select_region(filtered, 50)

In [ ]:
good = select_region(filtered, 8)

In [ ]:
spherality(good)

In [ ]:
spherality(wonky)

In [ ]:
print(np.unique(filtered))

In [ ]:
bad = [l for l in range(3, 51) if spherality(select_region(filtered, l)) < 0.5]

In [ ]:
bad

In [ ]:
select_region(filtered, 3).view(Image3D)

In [ ]:
for l in bad:
    filtered[np.where(filtered == l)] = 0

In [ ]:
filtered.view(Segmentation3D)

In [ ]:
larger_seg = Segmentation3D.from_file('root01segmentation.tif')

In [ ]:
larger_seg

In [ ]:
def filter_by_sphericity(segmentation):
    
    filtered = segmentation.copy()
    
    ids = set(np.unique(segmentation)) - set([0])

    bad = [l for l in ids if spherality(select_region(segmentation, l)) < 0.5]
    
    for l in bad:
        filtered[np.where(segmentation == l)] = 0
        
    return filtered

In [ ]:
new_filtered = filter_by_sphericity(larger_seg)

In [ ]:
new_filtered

In [ ]:
labels = set(np.unique(new_filtered)) - set([0])

In [ ]:
spheralities = [spherality(select_region(new_filtered, l)) for l in labels]

In [ ]:
spheralities

In [ ]:
eroded_sps = [spherality(binary_erosion(select_region(new_filtered, l))) for l in labels]

In [ ]:
for s1, s2, s3 in zip(dilated_sps, spheralities, eroded_sps):
    print(s1, s2, s3)

In [ ]:
from scipy.ndimage.morphology import binary_dilation

In [ ]:
dilated_sps = [spherality(binary_dilation(select_region(new_filtered, l))) for l in labels]